In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/atharvabhide/Quora-Question-Pair-Similarity/refs/heads/main/notebooks/train.csv")
df.sample(5)


,id,qid1,qid2,question1,question2,is_duplicate
14704,14704,28148,28149,Why do ideas appear easier while taking a shower?,Why do I get the best ideas while showering?,1
103565,103565,171158,43344,How much data can the human brain store? Is th...,Is there an ultimate limit to how much informa...,1
14194,14194,27205,27206,Is Trump still going to build a wall a make Me...,How is Trump planning to get Mexico to pay for...,1
20644,20644,38931,38932,What does it mean when a guy smiles at you eve...,What does it mean when a girl smiles at you ev...,0
231420,231420,341222,341223,Do men only want sex?,Why do men only want sex from women and not ma...,0


In [3]:
df.shape


(404290, 6)

In [4]:
new_df = df.sample(30000)


In [5]:
new_df.isnull().sum()


id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [6]:
new_df.duplicated().sum()


np.int64(0)

In [7]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()


,question1,question2
49117,Is there a way to use Google Voice Recognition...,How is Google's voice recognition so good?
329911,How do you find NAIC number for Geico Insurance?,How are State Farm and Geico different from ea...
168906,Is there any way I can track a lost iPhone 4s ...,How do I find my lost phone using imei number ...
230263,Are Hijras are transsexuals or intersex people?,Why do most people tolerate the rudeness of Hi...
180233,How can I learn to play music instrument while...,How can I learn music while working as a full ...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(), 2)


In [9]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)

temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.sample(5)


,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
234478,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
310526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
temp_df.shape


(30000, 6000)

In [ ]:
temp_df['is_duplicate'] = new_df['is_duplicate']
temp_df.sample()


,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
239909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:, 0:-1].values, temp_df.iloc[:, -1].values, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)


In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)
